In [ ]:
%load_ext lab_black

In [ ]:
from mlxtend.regressor import StackingCVRegressor
from skopt import BayesSearchCV
from sklearn.model_selection import RepeatedKFold
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
import pandas as pd
import os
from pyprojroot import here
import numpy as np
from skimpy import clean_columns
from data_cleaning.fun_hot_encode_limit import fun_hot_encode_limit
from pmdarima.model_selection import train_test_split
import optuna
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.model_selection import TimeSeriesSplit, cross_val_score
import joblib

path_data = here("./data")
os.chdir(path_data)
data_iu_analysis = pd.read_parquet("data_iu_analysis.parquet")

In [ ]:
print(xgb.__version__)
print(joblib.__version__)

In [95]:
test_size = round(data_iu_analysis.shape[0] * 0.2)

In [79]:
data_iu_analysis.columns

Index(['w', 'l', 'w_c', 'l_c', 'sos', 'ps_g', 'pa_g', 'ap_pre', 'ap_final',
       'tournament_make'],
      dtype='object')

In [97]:
train, test = train_test_split(data_iu_analysis, test_size=test_size)

In [98]:
x_train = train.drop(columns="tournament_make")
x_test = test.drop(columns="tournament_make")
y_train = train["tournament_make"]
y_test = test["tournament_make"]

In [120]:
# Define objective function for Optuna optimization
def objective(trial):
    params = {
        "objective": "binary:logistic",
        "metric": "binary_logloss",
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    # Set up TimeSeriesSplit cross-validation
    tscv = TimeSeriesSplit(n_splits=5)

    # Train XGBoost model with the given hyperparameters using cross-validation
    model = xgb.XGBClassifier(**params)
    scores = cross_val_score(model, x_train, y_train, cv=tscv, scoring="roc_auc")
    auc = scores.mean()

    return auc


# Run hyperparameter optimization using Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2023-04-14 17:03:03,127] A new study created in memory with name: no-name-55d85fc2-da17-4089-a26a-80dac760d757
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecat

In [119]:
# Train final XGBoost model using the best hyperparameters found by Optuna
best_params = study.best_params
best_model = xgb.XGBRegressor(**best_params)
best_model.fit(x_train, y_train)

# Return the best model
best_model

ValueError: No trials are completed yet.

In [ ]:
path_outputs = here("./outputs")
os.chdir(path_outputs)
joblib.dump(best_model, "model_iu_bball.jlib")

In [ ]:
model_iu_bball = joblib.load("model_iu_bball.jlib")

In [ ]:
prob = model_iu_bball.predict(x_test)

In [ ]:
from sklearn.metrics import f1_score


def find_best_threshold(y_true, y_pred_prob):
    """
    Find the best threshold to use for converting predicted probabilities to binary predictions,
    using the F1 score as the metric to optimize for.

    :param y_true: true labels, array-like of shape (n_samples,)
    :param y_pred_prob: predicted probabilities, array-like of shape (n_samples,)
    :return: best threshold value
    """
    thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    best_f1 = 0
    best_threshold = 0
    for threshold in thresholds:
        y_pred = (y_pred_prob >= threshold).astype(int)
        f1 = f1_score(y_true, y_pred)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    return best_threshold

In [ ]:
threshold = find_best_threshold(y_true=y_test, y_pred_prob=prob)
threshold

In [ ]:
prob_threshold = np.where(prob > threshold, 1, 0)

In [ ]:
from sklearn.metrics import balanced_accuracy_score, f1_score

In [ ]:
balanced_accuracy = balanced_accuracy_score(y_true=y_test, y_pred=prob_threshold)
balanced_accuracy

In [ ]:
f1_accuracy = f1_score(y_true=y_test, y_pred=prob_threshold)
f1_accuracy